In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score

#                                              Introduction (Why needed Stochastic Gradient Descent?)
```js
        The Stochastic Gradient Descent is used here is for Multiple Linear Regression where we have m Input Columns and obviously 1 Output Column. To understand this Gradient Descent, first have a CONSPICUOUS GRASP on 'Batch Gradient Descent' i.e. the Vanilla Gradient Descent.

        Purpose :
        ---------
        Minimizing the Loss Function finding the 'minima'. In our case, minimizing the Linear Regression''s Loss Function (MSE) finding the 'minima'.
        
        The 'Batch Gradient Descent' algo''s main calculation part for intercept and coefficients :
        -------------------------------------------------------------------------------------------

        def fit(self, x_train: np.ndarray, y_train: np.ndarray):
        B0 = 0 // -> Intercept.
        B = np.ones(shape = (x_train.shape[1], 1)) // B1, B2, B3 ..., B10 = [ [1], [1], [1], ..., [1] ] -> Coefficient. (10, 1).
        n = x_train.shape[0] // 353, Row numbers.       m = x_train.shape[1] = 10 columns.

        for i in range(self.epochs):
            y_hat = B0 + np.dot(x_train, B) // (353, 1) shape. y_hat = y_hat_1, y_hat_2 ..., y_hat_m.
            slope_B0 = (-2 / n) * np.sum(y_train - y_hat) // Scaler value.
            slope_Bs = (-2 / n) * np.dot((y_train - y_hat).T, x_train) // (1, 10) shape.
            slope_Bs = slope_Bs.reshape(slope_Bs.shape[::-1]) // (10, 1). B and slope_Bs must be in the same (_, 1) shape.

            B0 = B0 - self.learning_rate * slope_B0 // Scaler Value.
            B  = B  - self.learning_rate * slope_Bs // (10, 1) shape.

        self.intercept_, self.coef_ = B0, B
        print(f"Coefficients = \n{self.coef_}, \n\nIntercept = {self.intercept_}\n")

        The insights and issues in the above code :
        -------------------------------------------
        1) In 'Batch Gradient Descent' inside each epoch we update ALL THE COEFFICIENTS(B) with the help of vectorization usingg numpy dot matrix multiplication.
                i)   But before updating the coefficients(B) and intercept(B0), we need to calculate the 'y_hat'(y).
                ii)  To calculate y_hat we do especially 'np.dot(x_train, B)' i.e. we need to traverse all the rows of x_train to perform multiplication with B. Hold on! We also need to traverse all the rows of y_train in 'slope_B0 and slope_Bs'.
                iii) So to update all the coefficients(B1, B2 .. Bm) and intercept(B0) JUST FOR ONE TIME(which occurs in ONE EPOCH), WE NEED TO TRAVERSE ALL THE ROWS OF OUR WHOLE DATASET(x_train, y_train).

                iv) So basically 'Batch Gradient Descent' is : For each loop/epoch, traverse all the rows by another loop (i.e. vectorization) and after the nested loop, compute the coefficients and intercept.
        
        2) But in 'Stochastic Gradient Descent', inside the nested loop (which runs for each row of x_train) we update the coefficients(B1, B2 .. Bm) and intercept(B0) instant for the current row.
                i)  Then whats the difference with Batch Gradient Descent here? The difference is, yes we run a nested loop for row_numbers_of_x_train times and inside that loop "we randomly pick a row_index and update coefficients and intercept only for that random row".
                ii) Why randomly pick a row and update it but not pick row from the first serially? Watch (https://youtu.be/V7KBAa_gh4c?si=xS5UeBNNcjQbxjYf&t=2135).
                iii) For 100 epochs, Batch Gradient Descent updates coefficients and intercept for 100 times which works better for small to medium dataset. But for 100 epochs, Stochastic Gradient Descent updates coefficients and intercept for 100(outer loop) * 300(inner loop, assume we have 300 rows) = 3_00000 times which we need for larger dataset.

                iii) Since in Stochastic Gradient Descent, for less epochs we are updating the coefficients(B1, B2 .. Bm) and intercept(B0) A LOOOOOOT. That is why for Larger Dataset, Stochastic Gradient Descent helps us to reach the goal point in less time/epochs because for the same amount of updates we need more epochs in Batch Gradient Descent which is slower and impractical.
```

#                                                   Implementation Steps
```js
        The same steps like 'Batch Gradient Descent' but we now traverse all the rows of the Input Dataset(x_train, y_train) "MANUALLY" and update the coefficients and intercept for the randomly_picked_row.

        The first rows we need to update of Batch Gradient Descent are :

                    for i in range(self.epochs):
                        y_hat = B0 + np.dot(x_train, B) // (353, 1) shape. y_hat = y_hat_1, y_hat_2 ..., y_hat_m.
                        slope_B0 = (-2 / n) * np.sum(y_train - y_hat) // Scaler value.
                        slope_Bs = (-2 / n) * np.dot((y_train - y_hat).T, x_train) // (1, 10) shape.

        Changes for Stochastic Gradient Descent :
        -----------------------------------------
                    B0 = 0 // -> Intercept.
                    B = np.ones(shape = (x_train.shape[1])) // B1, B2, B3 ..., B10 = [1, 1, 1, ..., 1] -> Coefficient. (10).

                    for i in range(self.epochs):
                        for j in range(total_row_numbers):
                            row_idx = np.random.randint(low=0, high=total_row_numbers)
                            ..........................................................
        Now,                  
        1) B0 and B will be as it is since we are updating the whole B0 and B for the current randomly_picked_row i.e. 'row_idx'.

        2) For 'y_hat' : x_train here will be x_train_for_row_idx i.e. one row and B is also one row. np.dot(one_row, one_row) returns a "Scaler Value". So "y_hat is a scaler value".

                                    y_hat = B0 + np.dot(x_train[row_idx], B) // returns scaler value.

        3) For 'slope_B0' : We are updating coefficients and intercept for 1 row i.e. n = 1. y_train here will be y_train_for_row_idx i.e. a scaler value because y_train is 1D Array. So y_train - y_hat returns a scaler value, no need to do 'np.sum'.

                                    slope_B0 = -2 * (y_train - y_hat) // returns scaler value.
        
        4) For 'slope_Bs' : n = 1. (y_train - y_hat) is 'scaler value', so no need to do Transpose. x_train is 1D Array here. "Between a Scaler Value and 1D Array, np.dot(scaler, 1D_Array) and (scaler * 1D_Array) return same output.

                                    slope_Bs = -2 * ((y_train[row_idx] - y_hat) * x_train[row_idx])

```

#                                                     Code Implementation

In [ ]:
X, Y = load_diabetes(return_X_y=True) # returns np.ndarray. X.shape = (442, 10), Y.shape = (442,).

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2) # (353, 10), (89, 10), (353,), (89,).

lr = LinearRegression()
lr.fit(X = x_train, y = y_train) # calculated m, b for the future prediction.
m, b = lr.coef_, lr.intercept_
print(f"Coefficients = \n{m.reshape(m.shape[::-1])}, \n\nIntercept = {b}\n")

print(f"Predictions for the First 5 values = \n{lr.predict(x_test[:5])}\n")

print(f"r2 score = {r2_score(y_true=y_test, y_pred=lr.predict(x_test))}")

(353,)
Coefficients = 
[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238], 

Intercept = 151.88331005254167

Predictions for the First 5 values = 
[154.1213881  204.81835118 124.93755353 106.08950893 258.5348576 ]

r2 score = 0.4399338661568968


In [ ]:
class GDRegressor:
    def __init__(self, learning_rate: float, epochs: int):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
    
    def fit(self, x_train: np.ndarray, y_train: np.ndarray):
        B0 = 0 # -> Intercept.
        B = np.ones(shape = (x_train.shape[1])) # B1, B2, B3 ..., B10 = [1, 1, 1, ..., 1] -> Coefficient. (10).
        rows = x_train.shape[0] # 353, Total Row Numbers.

        for _ in range(self.epochs):
            for _ in range(rows):
                # Shape : x_train[row_idx] = (10), B = (10), y_train[row_idx] = Scaler value.
                row_idx = np.random.randint(low=0, high=rows)
                y_hat = B0 + np.dot(x_train[row_idx], B) # y_hat = Scaler Value.
                
                slope_B0 = -2 * (y_train[row_idx] - y_hat) # Scaler value.
                slope_Bs = -2 * (y_train[row_idx] - y_hat) * x_train[row_idx] # (x_train.shape[1]) i.e.
                                                                              # (input_column_numbers).
                B0 = B0 - self.lr * slope_B0 # Scaler Value.
                B  = B  - self.lr * slope_Bs # (input_column_numbers i.e. 353).

        self.intercept_, self.coef_ = B0, B
        print(f"Coefficients = \n{self.coef_}, \n\nIntercept = {self.intercept_}\n")
    
    def predict(self, x_test: np.ndarray):
        """
        x_test.shape = (5, 10) and self.coef_.shape = (10). np.dot(x_test.T, self.coef_).shape = (5).
        np.dot(x_test, self.coef_) + self.intercept_ = shape (5).
        """
        return np.dot(x_test, self.coef_) + self.intercept_ 

def main():
    gdr = GDRegressor(learning_rate = 0.1, epochs = 100)
    gdr.fit(x_train, y_train)

    print(f"Predictions for the First 5 values = \n{gdr.predict(x_test[:5])}\n")

    print(f"r2 score = {r2_score(y_true=y_test, y_pred=gdr.predict(x_test))}")

if __name__ == "__main__":
    main()

explanation = """
Sklearn returned :
------------------
Coefficients =
[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238], 

Intercept = 151.88331005254167

Predictions for the First 5 values = 
[154.1213881  204.81835118 124.93755353 106.08950893 258.5348576 ]

r2 score = 0.4399338661568968

Note : Since we're picking the row_number randomly, the output varies each time we run it. To optimize the output here, we
       use "Learning Schedule" which is (we use this "Learning Schedule" mostly in Deep Learning)

        learning_rate = lambda t: 5 / (t + 50)
        for _ in range(self.epochs):
            for _ in range(rows):
                self.lr = learning_rate(i * x_train.shape[0] + j)
                ....................
                ....................
"""

Coefficients = 
[   7.74291329 -214.29328175  508.94466724  321.74232391 -121.14533128
  -52.63431116 -166.32402329   63.45192667  596.53258923   42.95844928], 

Intercept = 155.46829576601914

Predictions for the First 5 values = 
[157.38705885 207.04744554 127.35619616 107.13719189 273.00344203]

r2 score = 0.44198659984993227


#                                                       The Rest Theory
```js
        Just a fancy tag, nothing else. Must watch [https://youtu.be/V7KBAa_gh4c?si=PCFgVHCgtF781F2w&t=1674] from 27:54 till end.
```

#                                       Sklearn's Stochastic Gradient Descent

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score

from sklearn.linear_model import SGDRegressor

In [119]:
X, Y = load_diabetes(return_X_y=True) # returns np.ndarray. X.shape = (442, 10), Y.shape = (442,).
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2) # (353, 10), (89, 10), (353,), (89,).

sgd = SGDRegressor(loss = 'squared_error', max_iter = 111, learning_rate = 'constant', eta0 = 0.01)
"""
loss = In our case, we are optimizing the Linear Regression's Loss Function which is MSE (Mean Squared Error).
max_iter = Maximum Epoch.
learning_rate = Want to keep the Initial Learning Rate constant or not.
eta0 = Initial Learning Rate.
"""

sgd.fit(X = x_train, y = y_train)

print(f"Coefficients = {sgd.coef_}\nIntercept = {sgd.intercept_}.\n")

print(f"r2 score = {r2_score(y_true = y_test, y_pred = sgd.predict(x_test))}.\n")

print(f"Prediction for the first 5 values = {sgd.predict(x_test[:5])}.")

warning = """
If we get the error "ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing
max_iter to improve the fit.", then need to increase the max_iter.
"""

Coefficients = [  60.86961237  -42.20575615  303.18632103  218.89600798   31.17966896
   -5.58495197 -158.80328801  128.3211942   279.31828638  126.17902842]
Intercept = [150.98436032].

r2 score = 0.4165326502819473.

Prediction for the first 5 values = [153.18706955 186.97492584 141.9537144  111.13859457 234.14998623].
